In [2]:
import numpy as np, pandas as pd, joblib
from pathlib import Path
from sklearn.metrics import precision_recall_fscore_support

# --- Rutas ---
DF_PATH   = "data/interim/ES_5m_2021_2024.parquet"
DATA_PATH = "data/processed/features/supervised_EUUSA.parquet"
EV_PATH   = "data/processed/events/events_labeled_2021_2024_EUUSA.parquet"
MODEL     = "models/artifacts/HGB_EUUSA_f23_calibrated_isotonic.joblib"

# --- Carga base ---
df        = pd.read_parquet(DF_PATH)
D         = pd.read_parquet(DATA_PATH)     # X + idx + zone_type + target
ev_euusa  = pd.read_parquet(EV_PATH)       # idx, zone_type, level_price, side_at_event, label,...

# Split temporal (2021-2023 train, 2024H1 val, 2024H2 test)
t_series  = df["Time"].dt.tz_convert("Europe/Madrid")
idx_ser   = D["idx"].astype(int)
TR = (t_series.loc[idx_ser] < "2024-01-01").values
VA = (t_series.loc[idx_ser] >= "2024-01-01").values & (t_series.loc[idx_ser] < "2024-07-01").values
TE = (t_series.loc[idx_ser] >= "2024-07-01").values

# Matriz de features X y etiquetas
X = D.drop(columns=["target","idx","zone_type"], errors="ignore").copy()
num = X.select_dtypes(include=[np.number]).columns
X[num] = X[num].fillna(0.0).astype(np.float32)
y      = D["target"].astype(str).values
zones  = D["zone_type"].astype(str).values
idxs   = D["idx"].astype(int).values

# Modelo calibrado (isotónica)
model   = joblib.load(MODEL)
classes = model.classes_

# Probabilidades en validación y test
Pva, yva, zva, iva = model.predict_proba(X[VA]), y[VA], zones[VA], idxs[VA]
Pte, yte, zte, ite = model.predict_proba(X[TE]), y[TE], zones[TE], idxs[TE]

def decide_rebound_only(P, classes, t_event, t_none, t_r, m_rb=0.0):
    """Predicción binaria {'rebound','none'} con gate evento y margen contra breakout."""
    c2i = {c:i for i,c in enumerate(classes)}
    pr, pb, pn = P[:,c2i["rebound"]], P[:,c2i["breakout"]], P[:,c2i["none"]]
    event_conf = np.maximum(pr, pb)
    yhat = np.where(
        (event_conf >= t_event) & (pn <= t_none) & (pr >= t_r) & ((pr - pb) >= m_rb),
        "rebound", "none"
    )
    return yhat

ZONES = ["PDH_prev","PDL_prev","USA_IBH","USA_IBL","VWAP","POC_D1","VAH_D1","VAL_D1"]
best_by_zone = {}

# Requisito: recall mínimo para 'rebound' (p.ej. 20%) para evitar colapsar recall
RECALL_MIN = 0.20

for z in ZONES:
    m = (zva == z)
    if m.sum() == 0:
        continue
    Pz, yz = Pva[m], yva[m]
    yz_bin = np.where(yz == "rebound", "rebound", "none")

    best = None
    for t_event in np.linspace(0.50, 0.75, 6):
        for t_none in np.linspace(0.35, 0.55, 5):
            for t_r in np.linspace(0.60, 0.80, 9):
                for m_rb in (0.00, 0.05, 0.10):
                    yhat = decide_rebound_only(Pz, classes, t_event, t_none, t_r, m_rb)
                    # métricas por clase en orden ['rebound','none']
                    P_,R_,F_,S_ = precision_recall_fscore_support(
                        yz_bin, yhat, labels=["rebound","none"], zero_division=0
                    )
                    f1_macro = float(F_.mean())
                    rec_reb  = float(R_[0])
                    # aplica restricción de recall y usa f1_macro como objetivo
                    if rec_reb >= RECALL_MIN:
                        cand = (f1_macro, rec_reb, (t_event,t_none,t_r,m_rb))
                        if (best is None) or (cand > best):
                            best = cand
    # fallback si nada cumple recall_min: coge el mejor f1_macro sin restricción
    if best is None:
        best = (0.0, 0.0, (0.55, 0.45, 0.65, 0.05))
    best_by_zone[z] = {
        "f1_macro_val": best[0],
        "recall_rebound_val": best[1],
        "params": {
            "t_event": best[2][0], "t_none": best[2][1],
            "t_r": best[2][2],     "m_rb":   best[2][3],
        }
    }

best_by_zone


c:\Users\jmbf2\anaconda3\envs\ML\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but HistGradientBoostingClassifier was fitted without feature names
  warnings.warn(
c:\Users\jmbf2\anaconda3\envs\ML\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but HistGradientBoostingClassifier was fitted without feature names
  warnings.warn(


{'PDH_prev': {'f1_macro_val': 0.6118868240415203,
  'recall_rebound_val': 0.532608695652174,
  'params': {'t_event': 0.6, 't_none': 0.55, 't_r': 0.6, 'm_rb': 0.1}},
 'PDL_prev': {'f1_macro_val': 0.5346938775510204,
  'recall_rebound_val': 0.32432432432432434,
  'params': {'t_event': 0.6, 't_none': 0.55, 't_r': 0.6, 'm_rb': 0.1}},
 'USA_IBH': {'f1_macro_val': 0.6584305791672116,
  'recall_rebound_val': 0.5818181818181818,
  'params': {'t_event': 0.6, 't_none': 0.55, 't_r': 0.625, 'm_rb': 0.1}},
 'USA_IBL': {'f1_macro_val': 0.6616117307933325,
  'recall_rebound_val': 0.6339285714285714,
  'params': {'t_event': 0.6, 't_none': 0.55, 't_r': 0.6, 'm_rb': 0.1}},
 'VWAP': {'f1_macro_val': 0.5465786711527125,
  'recall_rebound_val': 0.39473684210526316,
  'params': {'t_event': 0.6, 't_none': 0.55, 't_r': 0.6, 'm_rb': 0.1}},
 'POC_D1': {'f1_macro_val': 0.6151702204059298,
  'recall_rebound_val': 0.5303030303030303,
  'params': {'t_event': 0.6, 't_none': 0.55, 't_r': 0.6, 'm_rb': 0.1}},
 'VAH_D1'

In [4]:
# === Bloque 2) Evaluación en TEST con umbrales por zona (+ filtro ATR opcional) ===
import numpy as np, pandas as pd, joblib
from pathlib import Path
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix

# --- rutas (ajusta si usas otras) ---
DF_PATH   = "data/interim/ES_5m_2021_2024.parquet"
DATA_PATH = "data/processed/features/supervised_EUUSA.parquet"
MODEL     = "models/artifacts/HGB_EUUSA_f23_calibrated_isotonic.joblib"

# --- decide() binario (rebound vs none), mismo que en bloque 1 ---
def decide_rebound_only(P, classes, t_event, t_none, t_r, m_rb=0.0):
    c2i = {c:i for i,c in enumerate(classes)}
    pr, pb, pn = P[:,c2i["rebound"]], P[:,c2i["breakout"]], P[:,c2i["none"]]
    event_conf = np.maximum(pr, pb)
    return np.where(
        (event_conf >= t_event) & (pn <= t_none) & (pr >= t_r) & ((pr - pb) >= m_rb),
        "rebound", "none"
    )

# --- carga base (si hace falta recomputar Pte/yte/zte/ite/classes) ---
need_probs = any(k not in locals() for k in ["Pte","yte","zte","ite","classes","df"])
if need_probs:
    df  = pd.read_parquet(DF_PATH)
    D   = pd.read_parquet(DATA_PATH)
    t   = df["Time"].dt.tz_convert("Europe/Madrid")
    idx = D["idx"].astype(int)
    TE  = (t.loc[idx] >= "2024-07-01").values

    X = D.drop(columns=["target","idx","zone_type"], errors="ignore").copy()
    num = X.select_dtypes(include=[np.number]).columns
    X[num] = X[num].fillna(0.0).astype(np.float32)

    y      = D["target"].astype(str).values
    zones  = D["zone_type"].astype(str).values
    model  = joblib.load(MODEL)
    classes= model.classes_

    Pte = model.predict_proba(X[TE])
    yte = y[TE]
    zte = zones[TE]
    ite = D["idx"].astype(int).values[TE]

# --- ATR filtro de régimen (opcional) ---
use_atr_filter = True
atr_q = 0.40      # cuantil recomendado (sube o baja según agresividad)
atr_at_idx = df["ATR_14"].loc[ite].to_numpy()
atr_thr = np.nanquantile(df["ATR_14"].values, atr_q) if np.isfinite(df["ATR_14"].values).any() else 0.0

# --- predicción binaria por zona ---
yhat_bin = np.array(["none"]*len(ite), dtype=object)

for z, conf in best_by_zone.items():
    params = conf["params"]
    m = (zte == z)
    if m.sum() == 0:
        continue
    yh = decide_rebound_only(Pte[m], classes, params["t_event"], params["t_none"], params["t_r"], params["m_rb"])
    if use_atr_filter:
        keep = atr_at_idx[m] >= atr_thr
        yh = np.where(keep, yh, "none")
    yhat_bin[m] = yh

# --- métricas TEST (rebote vs none) ---
ytrue_bin = np.where(yte=="rebound","rebound","none")
P_,R_,F_,S_ = precision_recall_fscore_support(ytrue_bin, yhat_bin, labels=["rebound","none"], zero_division=0)
cm = confusion_matrix(ytrue_bin, yhat_bin, labels=["rebound","none"])

print("TEST – precision/recall/F1 por clase [rebound, none]")
print("precision:", [round(x,3) for x in P_])
print("recall   :", [round(x,3) for x in R_])
print("F1       :", [round(x,3) for x in F_])
print("support  :", S_.tolist())
print("\nMatriz de confusión (rows=true, cols=pred) [rebound, none]:\n", cm)

# --- cobertura de señales (cuántas 'rebound' lanzamos) ---
signals = (yhat_bin == "rebound").sum()
print(f"\nSeñales emitidas (rebound): {signals} de {len(yhat_bin)} ejemplos TEST "
      f"({signals/len(yhat_bin):.1%}). ATR filter = {use_atr_filter} @ Q{int(atr_q*100)}")

# --- resumen por zona (conteo & tasa de señal) ---
by_zone = pd.DataFrame({"zone_type": zte, "ytrue": ytrue_bin, "yhat": yhat_bin})
summary_zone = (by_zone.assign(sig = (by_zone["yhat"]=="rebound").astype(int))
                        .groupby("zone_type")
                        .agg(samples=("yhat","size"),
                             signals=("sig","sum"))
                        .assign(signal_rate=lambda d: d["signals"]/d["samples"])
                        .sort_values("samples", ascending=False))
print("\nResumen por zona (TEST):")
print(summary_zone.head(10).round(3))

# --- guardar umbrales por zona para reproducibilidad ---
Path("models/artifacts").mkdir(parents=True, exist_ok=True)
pd.DataFrame(best_by_zone).T.to_json("models/artifacts/rebound_thresholds_by_zone.json",
                                     orient="index", indent=2)
print("\nGuardado: models/artifacts/rebound_thresholds_by_zone.json")


TEST – precision/recall/F1 por clase [rebound, none]
precision: [0.653, 0.511]
recall   : [0.248, 0.856]
F1       : [0.36, 0.64]
support  : [990, 908]

Matriz de confusión (rows=true, cols=pred) [rebound, none]:
 [[246 744]
 [131 777]]

Señales emitidas (rebound): 377 de 1898 ejemplos TEST (19.9%). ATR filter = True @ Q40

Resumen por zona (TEST):
           samples  signals  signal_rate
zone_type                               
VWAP           432       61        0.141
USA_IBH        315       73        0.232
USA_IBL        294       66        0.224
POC_D1         224       49        0.219
PDH_prev       209       37        0.177
VAH_D1         200       37        0.185
VAL_D1         142       40        0.282
PDL_prev        82       14        0.171

Guardado: models/artifacts/rebound_thresholds_by_zone.json


c:\Users\jmbf2\anaconda3\envs\ML\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but HistGradientBoostingClassifier was fitted without feature names
  warnings.warn(


In [6]:
import numpy as np, pandas as pd, joblib, json
from pathlib import Path

# --- rutas ---
DF_PATH   = "data/interim/ES_5m_2021_2024.parquet"
DATA_PATH = "data/processed/features/supervised_EUUSA.parquet"
MODEL     = "models/artifacts/HGB_EUUSA_f23_calibrated_isotonic.joblib"
TH_PATH   = "models/artifacts/rebound_thresholds_by_zone.json"

# --- carga base y split TEST ---
df  = pd.read_parquet(DF_PATH)
D   = pd.read_parquet(DATA_PATH)
t   = df["Time"].dt.tz_convert("Europe/Madrid")
idx = D["idx"].astype(int)
TE  = (t.loc[idx] >= "2024-07-01").values

X = D.drop(columns=["target","idx","zone_type"], errors="ignore").copy()
num = X.select_dtypes(include=[np.number]).columns
X[num] = X[num].fillna(0.0).astype(np.float32)
yte  = D["target"].astype(str).values[TE]
zte  = D["zone_type"].astype(str).values[TE]
ite  = D["idx"].astype(int).values[TE]

model   = joblib.load(MODEL)
classes = model.classes_
# (truco para quitar el warning de sklearn)
Pte = model.predict_proba(X[TE].to_numpy())

# --- funciones ---
def decide_rebound_only(P, classes, t_event, t_none, t_r, m_rb=0.0):
    c2i = {c:i for i,c in enumerate(classes)}
    pr, pb, pn = P[:,c2i["rebound"]], P[:,c2i["breakout"]], P[:,c2i["none"]]
    event_conf = np.maximum(pr, pb)
    return np.where(
        (event_conf >= t_event) & (pn <= t_none) & (pr >= t_r) & ((pr - pb) >= m_rb),
        "rebound", "none"
    )

with open(TH_PATH, "r", encoding="utf-8") as f:
    best_by_zone = json.load(f)

# --- predicción binaria por zona + filtro ATR (igual que bloque 2) ---
use_atr_filter = True
atr_q = 0.40
atr_at_idx = df["ATR_14"].loc[ite].to_numpy()
atr_thr = np.nanquantile(df["ATR_14"].values, atr_q) if np.isfinite(df["ATR_14"].values).any() else 0.0

yhat_bin = np.array(["none"]*len(ite), dtype=object)
for z, conf in best_by_zone.items():
    p = conf["params"]; m = (zte == z)
    if m.sum()==0: continue
    yh = decide_rebound_only(Pte[m], classes, p["t_event"], p["t_none"], p["t_r"], p["m_rb"])
    if use_atr_filter:
        keep = atr_at_idx[m] >= atr_thr
        yh = np.where(keep, yh, "none")
    yhat_bin[m] = yh

# --- backtest ---
TICK      = 0.25
P_INVAL   = 6     # stop por invalidación (ticks)
TP_TICKS  = 12    # take profit (ticks)
H         = 12    # horizonte máx (barras)

# nivel y lado por evento (para calcular TP/SL desde el nivel)
ev = pd.read_parquet("data/processed/events/events_labeled_2021_2024_EUUSA.parquet") \
       [["idx","zone_type","level_price","side_at_event"]].drop_duplicates()

test_ev = pd.DataFrame({"idx": ite, "zone_type": zte, "ytrue": yte, "yhat": yhat_bin}) \
          .merge(ev, on=["idx","zone_type"], how="left")

def backtest_rebound(ev_rows, df, tp_ticks=TP_TICKS, p_inval=P_INVAL, H=H, tick=TICK):
    hi = df["High"].to_numpy(float); lo = df["Low"].to_numpy(float); op = df["Open"].to_numpy(float)
    out = []
    for _, r in ev_rows.iterrows():
        i = int(r["idx"]); 
        if i+1 >= len(df): continue
        lvl = float(r["level_price"]); side = str(r["side_at_event"])
        if side == "support":
            tp = lvl + tp_ticks*tick; sl = lvl - p_inval*tick; direction="long"
        else:
            tp = lvl - tp_ticks*tick; sl = lvl + p_inval*tick; direction="short"
        entry = float(op[i+1]); outcome="timeout"; exit_px=float(op[min(i+H,len(df)-1)]); pnl_ticks=0.0
        for j in range(i+1, min(i+H+1, len(df))):
            if direction=="long":
                if lo[j] <= sl: outcome="loss";  exit_px=sl;  pnl_ticks=(exit_px-entry)/tick; break
                if hi[j] >= tp: outcome="win";   exit_px=tp;  pnl_ticks=(exit_px-entry)/tick; break
            else:
                if hi[j] >= sl:
                    outcome = "loss"
                    exit_px = sl
                    pnl_ticks = (entry - exit_px) / tick  # NEGATIVO si SL>entry
                    break
                if lo[j] <= tp:
                    outcome = "win"
                    exit_px = tp
                    pnl_ticks = (entry - exit_px) / tick  # POSITIVO si TP<entry
                    break
        if outcome == "timeout":
            pnl_ticks = (exit_px - entry)/tick if direction=="long" else (entry - exit_px)/tick

        out.append({"idx":i,"zone_type":r["zone_type"],"side":side,"entry":entry,"tp":tp,"sl":sl,
                    "exit":exit_px,"outcome":outcome,"pnl_ticks":pnl_ticks})
    return pd.DataFrame(out)

signals = test_ev[test_ev["yhat"]=="rebound"].copy()
bt = backtest_rebound(signals, df, tp_ticks=TP_TICKS, p_inval=P_INVAL, H=H, tick=TICK)

# KPIs
n = len(bt)
win_rate = (bt["outcome"]=="win").mean() if n else 0.0
ev = bt["pnl_ticks"].mean() if n else 0.0
avg_win  = bt.loc[bt["outcome"]=="win","pnl_ticks"].mean()
avg_loss = bt.loc[bt["outcome"]=="loss","pnl_ticks"].mean()
print(f"Signals: {n} | Win%: {win_rate:.3f} | EV (ticks/trade): {ev:.2f}")
print(f"Avg win: {avg_win:.2f} | Avg loss: {avg_loss:.2f}")

print("\nPor zona (n, Win%, EV):")
per_zone = bt.groupby("zone_type")["pnl_ticks"].agg(['count','mean'])
per_zone["win_rate"] = bt.groupby("zone_type")["outcome"].apply(lambda s: (s=='win').mean())
print(per_zone.sort_values("count", ascending=False).round(3).head(10))

# --- sensibilidad rápida TP/SL/H ---
grid_tp = [10,12,16]
grid_sl = [6,8]
grid_h  = [10,12,16]
summary = []
for tp in grid_tp:
    for sl in grid_sl:
        for h in grid_h:
            tmp = backtest_rebound(signals, df, tp_ticks=tp, p_inval=sl, H=h, tick=TICK)
            if len(tmp)==0: 
                summary.append((tp,sl,h,0,0,0)); continue
            wr = (tmp["outcome"]=="win").mean()
            evg= tmp["pnl_ticks"].mean()
            summary.append((tp,sl,h,len(tmp),wr,evg))
            
sens = pd.DataFrame(summary, columns=["TP","SL","H","n","Win%","EV_ticks"]).sort_values(["EV_ticks","n"], ascending=False)
print("\nSensibilidad (top 10 por EV):")
print(sens.head(10).round(3))


Signals: 377 | Win%: 0.607 | EV (ticks/trade): -0.09
Avg win: 6.96 | Avg loss: -10.99

Por zona (n, Win%, EV):
           count   mean  win_rate
zone_type                        
USA_IBH       73  1.904     0.699
USA_IBL       66  0.197     0.621
VWAP          61  0.041     0.607
POC_D1        49  0.184     0.633
VAL_D1        40 -2.600     0.475
PDH_prev      37 -1.595     0.541
VAH_D1        37 -0.432     0.595
PDL_prev      14 -1.286     0.571

Sensibilidad (top 10 por EV):
    TP  SL   H    n   Win%  EV_ticks
17  16   8  16  377  0.581     0.964
15  16   8  10  377  0.570     0.956
16  16   8  12  377  0.576     0.892
14  16   6  16  377  0.528     0.630
12  16   6  10  377  0.520     0.598
13  16   6  12  377  0.525     0.593
9   12   8  10  377  0.655     0.195
11  12   8  16  377  0.658     0.134
10  12   8  12  377  0.655     0.110
6   12   6  10  377  0.607    -0.062


In [7]:
# === Rebound v0.1 – Backtest con whitelist de zonas y TP/SL/H recomendados ===
import json, pandas as pd, numpy as np, joblib
from pathlib import Path

# rutas y objetos ya conocidos
DF_PATH   = "data/interim/ES_5m_2021_2024.parquet"
DATA_PATH = "data/processed/features/supervised_EUUSA.parquet"
EV_PATH   = "data/processed/events/events_labeled_2021_2024_EUUSA.parquet"
MODEL     = "models/artifacts/HGB_EUUSA_f23_calibrated_isotonic.joblib"
TH_PATH   = "models/artifacts/rebound_thresholds_by_zone.json"

# params v0.1
ZONE_WHITELIST = {"USA_IBH","USA_IBL","VWAP","POC_D1","PDH_prev","VAH_D1"}  # excluye VAL_D1/PDL_prev
ATR_Q = 0.40
TP_TICKS, SL_TICKS, H = 16, 8, 16
TICK = 0.25

# carga base y split TEST
df  = pd.read_parquet(DF_PATH)
D   = pd.read_parquet(DATA_PATH)
t   = df["Time"].dt.tz_convert("Europe/Madrid")
idx = D["idx"].astype(int)
TE  = (t.loc[idx] >= "2024-07-01").values

X = D.drop(columns=["target","idx","zone_type"], errors="ignore").copy()
num = X.select_dtypes(include=[np.number]).columns
X[num] = X[num].fillna(0.0).astype(np.float32)

yte  = D["target"].astype(str).values[TE]
zte  = D["zone_type"].astype(str).values[TE]
ite  = D["idx"].astype(int).values[TE]

model   = joblib.load(MODEL)
classes = model.classes_
Pte     = model.predict_proba(X[TE].to_numpy())

with open(TH_PATH,"r",encoding="utf-8") as f:
    best_by_zone = json.load(f)

def decide_rebound_only(P, classes, t_event, t_none, t_r, m_rb=0.0):
    c2i = {c:i for i,c in enumerate(classes)}
    pr, pb, pn = P[:,c2i["rebound"]], P[:,c2i["breakout"]], P[:,c2i["none"]]
    event_conf = np.maximum(pr, pb)
    return np.where(
        (event_conf >= t_event) & (pn <= t_none) & (pr >= t_r) & ((pr - pb) >= m_rb),
        "rebound", "none"
    )

# pred binaria con whitelist + ATR filter
atr_at_idx = df["ATR_14"].loc[ite].to_numpy()
atr_thr = np.nanquantile(df["ATR_14"].values, ATR_Q)
yhat = np.array(["none"]*len(ite), dtype=object)

for z, conf in best_by_zone.items():
    if z not in ZONE_WHITELIST: 
        continue
    p = conf["params"]; m = (zte == z)
    if m.sum()==0: continue
    yh = decide_rebound_only(Pte[m], classes, p["t_event"], p["t_none"], p["t_r"], p["m_rb"])
    keep = atr_at_idx[m] >= atr_thr
    yhat[m] = np.where(keep, yh, "none")

# merge con eventos (nivel y lado) y filtra señales
ev = pd.read_parquet(EV_PATH)[["idx","zone_type","level_price","side_at_event"]].drop_duplicates()
test_ev = pd.DataFrame({"idx": ite, "zone_type": zte, "yhat": yhat}).merge(ev, on=["idx","zone_type"], how="left")
signals = test_ev[(test_ev["yhat"]=="rebound") & (test_ev["zone_type"].isin(ZONE_WHITELIST))].copy()

# backtest
hi = df["High"].to_numpy(float); lo = df["Low"].to_numpy(float); op = df["Open"].to_numpy(float)
rows=[]
for _, r in signals.iterrows():
    i=int(r["idx"]); 
    if i+1>=len(df): continue
    lvl=float(r["level_price"]); side=str(r["side_at_event"])
    if side=="support":
        tp=lvl+TP_TICKS*TICK; sl=lvl-SL_TICKS*TICK; direction="long"
    else:
        tp=lvl-TP_TICKS*TICK; sl=lvl+SL_TICKS*TICK; direction="short"
    entry=float(op[i+1]); outcome="timeout"; exit_px=float(op[min(i+H,len(df)-1)])
    pnl=0.0
    for j in range(i+1, min(i+H+1,len(df))):
        if direction=="long":
            if lo[j]<=sl: outcome="loss";  exit_px=sl; pnl=(exit_px-entry)/TICK; break
            if hi[j]>=tp: outcome="win";   exit_px=tp; pnl=(exit_px-entry)/TICK; break
        else:
            if hi[j]>=sl: outcome="loss";  exit_px=sl; pnl=(entry-exit_px)/TICK*(-1); break
            if lo[j]<=tp: outcome="win";   exit_px=tp; pnl=(entry-exit_px)/TICK; break
    if outcome=="timeout":
        pnl=(exit_px-entry)/TICK if direction=="long" else (entry-exit_px)/TICK
    rows.append({"idx":i,"zone_type":r["zone_type"],"side":side,"entry":entry,"tp":tp,"sl":sl,"exit":exit_px,"outcome":outcome,"pnl_ticks":pnl})

bt = pd.DataFrame(rows)
print(f"Signals: {len(bt)} | Win%: {(bt['outcome']=='win').mean():.3f} | EV (ticks/trade): {bt['pnl_ticks'].mean():.2f}")
print(bt.groupby('zone_type')['pnl_ticks'].agg(['count','mean']).assign(win=lambda s: bt.groupby('zone_type')['outcome'].apply(lambda x: (x=='win').mean())).sort_values('count', ascending=False).round(3).head(10))

# guarda trades y resumen
Path("experiments/runs").mkdir(parents=True, exist_ok=True)
bt.to_csv("experiments/runs/rebound_v01_trades_test.csv", index=False)
summary = {
    "zones": sorted(list(ZONE_WHITELIST)),
    "atr_quantile": ATR_Q,
    "tp_ticks": TP_TICKS,
    "sl_ticks": SL_TICKS,
    "h": H,
    "n_trades": int(len(bt)),
    "win_rate": float((bt['outcome']=='win').mean()),
    "ev_ticks": float(bt['pnl_ticks'].mean())
}
import json; json.dump(summary, open("experiments/runs/rebound_v01_summary.json","w"), indent=2)
print("Guardado: experiments/runs/rebound_v01_trades_test.csv, rebound_v01_summary.json")


Signals: 323 | Win%: 0.601 | EV (ticks/trade): 7.00
           count   mean    win
zone_type                     
USA_IBH       73  7.685  0.658
USA_IBL       66  6.727  0.606
VWAP          61  8.109  0.574
POC_D1        49  5.612  0.673
PDH_prev      37  6.892  0.514
VAH_D1        37  6.216  0.514
Guardado: experiments/runs/rebound_v01_trades_test.csv, rebound_v01_summary.json
